In [2]:
import requests
from bs4 import BeautifulSoup

baseUrl = 'https://www.wine.com'

page = requests.get(f'{baseUrl}/list/wine/red-wine/7155-124?showOutOfStock=true')

soup = BeautifulSoup(page.text, 'html.parser')


In [3]:
wineList = soup.find('ul', class_='listGridLayout_list')
wineElements = wineList.find_all('li', class_='listGridLayout_listItem')

In [59]:
# iterate <li> items
for wineElem in wineElements[:5]:
    # Try <5 times to load the product page
    trials = 0
    while trials < 5:
        try:
            # is out of stock?
            outOfStock = wineElem.find('div', class_='productUnavailable')

            wineInfo = { 'color': 'red' }   # Iterate over red, white, ...

            # ----- Getting info from main page -----
            # Name and vintage  
            name = wineElem.find('span', itemprop='name').text
            vintage = name[-4:]
            wineInfo['name'] = name
            wineInfo['vintage'] = int(vintage)

            # Varietal
            varietal = wineElem.find('span', class_='listGridItemOrigin_varietal').text
            wineInfo['varietal'] = varietal

            # Origin
            origin = wineElem.find('span', class_='listGridItemOrigin_text').text
            wineInfo['origin'] = origin

            # Price
            price = wineElem.find('meta', itemprop='price')['content']
            wineInfo['price'] = float(price)

            # Ratings
            ulRatings = wineElem.find('ul', class_='wineRatings_list')
            if ulRatings:
                ratings = []
                for ratingElem in ulRatings.find_all('li'):
                    ratingName = ratingElem['title'][:-16].strip()
                    ratingValue = ratingElem['title'][-15:-13].strip()
                    
                    rating = {'name': ratingName, 'rating': int(ratingValue)}

                    ratings.append(rating)

                wineInfo['ratings'] = ratings

            # ----- Getting info from wine page -----
            
            # Get wine detail URL
            a = wineElem.find('a', class_='event_productClick', href=True)
            wineUrl = f"{baseUrl}{a['href']}"

            # Open wine page
            winePage = requests.get(wineUrl)
            wineSoup = BeautifulSoup(winePage.text, 'html.parser')       

            if outOfStock: # If out of stock, page is different
                # Size
                size = wineSoup.find('span', class_='prodAlcoholVolume_text').text
                wineInfo['size'] = size

                # Images
                divImage = wineSoup.find('div', class_='pipThumbs')
                images = []
                for image in divImage.find_all('img'):
                    imageUrl = image['src'].split('/')[-1]
                    images.append('https://www.wine.com/product/images/w_600,h_600,c_fit,q_auto:good,fl_progressive/' + imageUrl)
            else:
                # Getting info from product details table
                productDetails = wineSoup.find('section', class_='pipProdDetails')
                titles = productDetails.find_all('div', class_='pipProdDetails_title')
                values = productDetails.find_all('div', class_='pipProdDetails_name')
                for i, title in enumerate(titles):
                    if title.text.strip() == 'Size':    # Size
                        wineInfo['size'] = values[i].text[:-2]
                    elif title.text.strip() == 'Producer':  # Producer
                        wineInfo['producer'] = values[i].text
                
                # Images
                divImage = wineSoup.find('div', class_='pipProdThumbs')
                images = []
                for image in divImage.find_all('img'):
                    imageUrl = image['src'].split('/')[-1]
                    images.append('https://www.wine.com/product/images/w_600,h_600,c_fit,q_auto:good,fl_progressive/' + imageUrl)
            
            wineInfo['image'] = images

            # Print wine
            print(wineUrl)
            for key, value in wineInfo.items():
                print(f"{key}: {value}")
            print()
            
            break
        except Exception as err:
            trials += 1
            continue
    else:
        print(f"FAIL {wineUrl}")

https://www.wine.com/product/altesino-brunello-di-montalcino-2017/877499
color: red
name: Altesino Brunello di Montalcino 2017
vintage: 2017
varietal: Sangiovese
origin: Montalcino, Tuscany, Italy
price: 74.99
ratings: [{'name': 'James Suckling', 'rating': 95}, {'name': "Robert Parker's Wine Advocate", 'rating': 94}, {'name': 'Wine Spectator', 'rating': 92}, {'name': 'Wine Enthusiast', 'rating': 90}]
producer: Altesino
size: 750
image: ['https://www.wine.com/product/images/w_600,h_600,c_fit,q_auto:good,fl_progressive/hhcpugypymo1t9rosdye.jpg', 'https://www.wine.com/product/images/w_600,h_600,c_fit,q_auto:good,fl_progressive/z89hjnmoryon9ij12ew7.jpg']

https://www.wine.com/product/le-cellier-des-princes-cotes-du-rhone-la-couronne-du-prince-2018/1079083
color: red
name: Le Cellier des Princes Cotes du Rhone La Couronne du Prince 2018
vintage: 2018
varietal: Rhone Red Blends
origin: Cotes du Rhone, Rhone, France
price: 15.98
ratings: [{'name': 'James Suckling', 'rating': 91}]
producer: Le

In [33]:
text = 'Antinori Badia a Passignano Chianti Classico Gran Selezione 2018'

print(text.replace('Antinori', ''))

 Badia a Passignano Chianti Classico Gran Selezione 2018
